In [1]:
import os
import scipy.io.wavfile as wf
import numpy as np
import random
import matplotlib.pyplot as plt

In [2]:
####
#
def norm(inp):
    M = np.amax(np.absolute(inp))
    return inp/M

def iSNR_control(signal,noise,SNR):
    
    #now A
    A_now = 0
    for i in noise :
        A_now = A_now + i*i
    A_now = np.sqrt(A_now/len(noise))
    
    #aim A
    A_sig = 0
    for i in signal :
        A_sig = A_sig + i*i
    A_sig = np.sqrt(A_sig/len(signal))
    
    A_aim = A_sig/(10**(SNR/20))
    
    k = A_aim/A_now
#     print(k)
    noise = k*noise
    return noise

def get_n_sample(signal,noiseR,noiseL):
    
    Ls = len(signal)
    Ln = len(noiseR)
    
    Ln = Ln-Ls
    p = random.randrange(0,Ln)
    
    return noiseR[p:p+Ls],noiseL[p:p+Ls]

#
####

In [3]:
NOISE_NAMES={0:'DLIVING',
             1:'DWASHING',
             2:'NFIELD',
             3:'NPARK',
             4:'NRIVER',
             5:'OHALLWAY',
             6:'OMEETING',
             7:'OOFFICE',
             8:'PCAFETER',
             9:'PRESTO',
             10:'PSTATION',
             11:'SPSQUARE',
             12:'STRAFFIC',
             13:'TBUS',
             14:'TCAR',
             15:'TMETRO'}


SNR_train_list = [15,10,5,0]
SNR_test_list = [17.5,12.5,7.5,2.5]
print(NOISE_NAMES[4])

NRIVER


In [4]:
LD = os.listdir('./clean_trainset_wav_16k')
I = []
t = True
while t:
    n1 = random.randrange(0,16)
    n2 = random.randrange(0,16)
    n3 = random.randrange(0,16)

    d1 = random.randrange(0,6)
    d2 = random.randrange(6,12)
    d3 = random.randrange(12,18)
      
    k = [n1,n2,n3,d1,d2,d3]

    I.append(k)
    if len(I) == len(LD):
        t = False

In [5]:
indx = 0
print(LD[0])
for f_name in LD[:5]:
    print(f_name)
    SR,clean_wav = wf.read('./clean_trainset_wav_16k/%s'%(f_name))
    clean_wav = norm(clean_wav)
#     print('clean_wav : ',type(clean_wav[0]))
    
    nn1,nn2,nn3,nd1,nd2,nd3 = I[indx]
    
    if nd1 >= 9:
        nd1 = nd1+1
    if nd2 >= 9:
        nd2 = nd2+1
    if nd3 >= 9:
        nd3 = nd3+1
        
    SR,noise1R = wf.read('./DEMAND_16k_4/%s_%dd_R.wav'%(NOISE_NAMES[nn1],nd1))
    if SR != 16000:
        print(f_name,' is WRONG')
    SR,noise1L = wf.read('./DEMAND_16k_4/%s_%dd_L.wav'%(NOISE_NAMES[nn1],nd1))
    if SR != 16000:
        print(f_name,' is WRONG')
        
#     SR,noise2R = wf.read('./DEMAND_16k_4/%s_%dd_R.wav'%(NOISE_NAMES[nn2],nd2))
#     if SR != 16000:
#         print(f_name,' is WRONG')
#     SR,noise2L = wf.read('./DEMAND_16k_4/%s_%dd_L.wav'%(NOISE_NAMES[nn2],nd2))
#     if SR != 16000:
#         print(f_name,' is WRONG')
        
#     SR,noise3R = wf.read('./DEMAND_16k_4/%s_%dd_R.wav'%(NOISE_NAMES[nn3],nd3))
#     if SR != 16000:
#         print(f_name,' is WRONG')
#     SR,noise3L = wf.read('./DEMAND_16k_4/%s_%dd_L.wav'%(NOISE_NAMES[nn3],nd3))
#     if SR != 16000:
#         print(f_name,' is WRONG')
        
    indx = indx+1
    
    n1R_sample,n1L_sample = get_n_sample(clean_wav,noise1R,noise1L)
    
#     n2R_sample,n2L_sample = get_n_sample(clean_wav,noise3R,noise2L)
    
#     n3R_sample,n3L_sample = get_n_sample(clean_wav,noise3R,noise3L)

    noiseR = n1R_sample#+n2R_sample+n3R_sample
    #noiseL = n1L_sample+n2L_sample+n3L_sample
    for iii in range(4):
        noiseR_ = iSNR_control(signal=clean_wav,noise=noiseR,SNR=SNR_train_list[iii])
        #noiseL = iSNR_control(signal=clean_wav,noise=noiseL,SNR=SNR_test_list[indx%4])

        noisy_wav1 = norm(np.array(clean_wav+noiseR_))
        #noisy_wav2 = norm(np.array(clean_wav+noiseL))

#         if len(noisy_wav1) != len(noisy_wav2):
#             raise ValueError
        if len(noisy_wav1) != len(clean_wav):
           raise ValueError

        noisy_wav1 = noisy_wav1.astype(np.float32)
        #noisy_wav2 = noisy_wav2.astype(np.float32)
        clean_wav = clean_wav.astype(np.float32)
        
        wf.write('./khj/%f_%s'%(SNR_train_list[iii],f_name),SR,noisy_wav1)
    wf.write('./khj/clean_%s'%f_name,SR,clean_wav)



#     wf.write('./DSEnet_test_DATA_1cm_4/MICR/%s'%f_name,SR,noisy_wav1)
    #wf.write('./DSEnet_test_DATA_1cm_4/MICL/%s'%f_name,SR,noisy_wav2)
#     wf.write('./DSEnet_test_DATA_1cm_4/LookD/%s'%f_name,SR,clean_wav)


p226_001.wav
p226_001.wav
p226_002.wav
p226_003.wav
p226_004.wav
p226_005.wav
